In [6]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Face Mesh and Drawing utilities
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# Create FaceMesh object
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True, max_num_faces=1)

# Open webcam
cap = cv2.VideoCapture(0)

# Eye blink detection thresholds
EYE_AR_THRESH = 0.22

def eye_aspect_ratio(landmarks, eye_indices):
    """Calculate Eye Aspect Ratio to detect blinking"""
    A = np.linalg.norm(landmarks[eye_indices[1]] - landmarks[eye_indices[5]])
    B = np.linalg.norm(landmarks[eye_indices[2]] - landmarks[eye_indices[4]])
    C = np.linalg.norm(landmarks[eye_indices[0]] - landmarks[eye_indices[3]])
    ear = (A + B) / (2.0 * C)
    return ear

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = face_mesh.process(rgb_frame)
    
    if result.multi_face_landmarks:
        for face_landmarks in result.multi_face_landmarks:
            # Custom green mesh drawing
            mp_drawing.draw_landmarks(
                image=frame,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing.DrawingSpec(
                    color=(0, 255, 0),  # Green mesh
                    thickness=1,
                    circle_radius=0
                )
            )

            # Green contours for clarity
            mp_drawing.draw_landmarks(
                image=frame,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_CONTOURS,
                landmark_drawing_spec=mp_drawing.DrawingSpec(
                    color=(0, 255, 0),
                    thickness=2,
                    circle_radius=1
                ),
                connection_drawing_spec=mp_drawing.DrawingSpec(
                    color=(0, 255, 0),
                    thickness=2,
                    circle_radius=1
                )
            )

            # Convert landmarks to numpy array
            landmarks = np.array([(lm.x * w, lm.y * h) for lm in face_landmarks.landmark])
            
            # Eye landmarks
            LEFT_EYE = [33, 160, 158, 133, 153, 144]
            RIGHT_EYE = [362, 385, 387, 263, 373, 380]
            
            # Eye blink detection
            left_ear = eye_aspect_ratio(landmarks, LEFT_EYE)
            right_ear = eye_aspect_ratio(landmarks, RIGHT_EYE)
            ear = (left_ear + right_ear) / 2.0
            
            if ear < EYE_AR_THRESH:
                cv2.putText(frame, "BLINK DETECTED", (50, 140), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 3)
            
            # Head movement detection
            nose_tip = landmarks[1]
            left_cheek = landmarks[234]
            right_cheek = landmarks[454]
            chin = landmarks[152]
            forehead = landmarks[10]
            
            # Horizontal movement
            if nose_tip[0] < left_cheek[0]:
                direction = "LEFT"
            elif nose_tip[0] > right_cheek[0]:
                direction = "RIGHT"
            else:
                direction = "CENTER"
            
            # Vertical movement
            vertical_diff = chin[1] - forehead[1]
            if vertical_diff > 180:
                vertical = "DOWN"
            elif vertical_diff < 130:
                vertical = "UP"
            else:
                vertical = "CENTER"
            
            # Display text on screen
            cv2.putText(frame, f"Horizontal: {direction}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 0), 3)
            cv2.putText(frame, f"Vertical: {vertical}", (50, 90), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 255), 3)

    # Show the output window
    cv2.imshow("Facial Recognition - Green Mesh + Blink + Head Movement", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
